In [1]:
import random
from numpy.random import rand, choice
import numpy as np
import pandas as pd

In [382]:
class skullPlayer:
    def __init__(self):
        self.flowers = 3       #GR          how many flowers the player currently has
        self.skull = 1         #GR          how many skulls the player currently has
        self.totalTiles = 4    #GR          how many tiles the player has in total
        self.inPlay = True     #GR          is the player still in the game 
        self.hasPoint = False  #GR          does the player have a point already
        self.externalHasSkull = True  #GR*  do the other players know for certain that the player has a skull
        self.externalHasNoSkull = False #GR do the other player know the player does not have a skull
        self.placedTiles = []  #RR          what tiles the player has currently placed
        self.hasPassed = False #RR          has the the player passed in this round of betting
        self.currentBet = None #RR          the current bet the player has made
        
        self.recPosition = None #GR TODO
        self.gameState = None #GR TODO
        self.betrec = None #RR
        self.game = None #GR TODO
        self.pickRec = None #RR
        
        self.theta_tp=(rand(1,30)*2)-1 #AI
        self.theta_bp=(rand(1,31)*2)-1 #AI
        self.theta_w2b=(rand(12,95)*2-1) #AI
        self.theta_bpa=(rand(1,95)*2-1) #AI
        self.theta_w2p=(rand(3,73)*2-1) #AI
        
    def removeTile(self):
        if self.inPlay:
            if self.skull==0:
                self.flowers+=-1
            else:
                if choice([True]+[False]*self.flowers):
                    self.skull+=-1
                else:
                    self.flowers+=-1

            self.totalTiles+=-1
            self.externalHasSkull = False

            if self.totalTiles==0:
                self.inPlay=False
                self.gameState.updateFirstPlayer((self.recPosition+1)%4)
            else:
                self.gameState.updateFirstPlayer(self.recPosition)
            self.game.resetRound()
        else:
            raise NameError("Player is not in play")
                
    def placeFlower(self):
        if self.inPlay & (self.flowers>0):
            if (len(self.placedTiles)-sum(self.placedTiles))<self.flowers:
                self.placedTiles.append(False)
                self.gameState.gatherInfo()
            else:
                raise NameError('All flowers have been placed')
        else:
            if not(self.inPlay):
                raise NameError("Player is not in play")
            else:
                raise NameError('No flowers to place')
    
    def placeSkull(self):
        if self.inPlay & (self.skull>0):
            if any(self.placedTiles):
                raise NameError('Skull has been placed')
            else:
                self.placedTiles.append(True)
                self.gameState.gatherInfo()
        else:
            if not(self.inPlay):
                raise NameError("Player is not in play")
            else:
                raise NameError('No Skull to place')
                
    def resetRound(self):
        self.placedTiles=[]
        self.hasPassed = False
        self.currentBet = None
        
    def resetGame(self):
        self.flowers=3
        self.skull=1
        self.totalTiles=4
        self.inPlay=True
        self.hasPoint = False
        self.externalHasSkull = True
        self.externalHasNoSkull = False
        
        self.recPosition = None
        self.gameState = None
        
        self.resetRound()
        
    def playerSummary(self):
        print("Flowers: {}\nSkull: {}\nPlaced tiles: {}".format(self.flowers,self.skull, self.placedTiles))
        
    def placeTile(self):
        if self.inPlay & (len(self.placedTiles)<self.totalTiles):
            if any(self.placedTiles)|(self.skull==0):
                self.placeFlower()
            elif ((len(self.placedTiles)-sum(self.placedTiles))==self.flowers):
                self.placeSkull()
            else:
                self.chooseTileToPlace()
        else:
            if not(self.inPlay):
                pass
            else:
                raise NameError('Max number of tiles placed')
            
    def chooseTileToPlace(self):
        """
        Function to decide what tile to place.
        External info should have for each player:
        - If they have a point (binary)
        - How many tiles they have (numerical)
        - How many tiles they have placed (numerical)
        - If they have their skull (dual-binary) [1,0] skull, [0,0] unknown, [0,1] no skull 
        - their place in the turn order (numerical)
        - if they're out of the game (binary) [1] out, [0] in
        - optional (not implemented)
            - if they just lost a tile
            - if they just won a point
        LAST row should be that on the "bot"
        Extra
        - random float for "randomness"
        - bias term
        therefore external info is a 1x30 vector
        
        RETURNS
        boolean. true, place a skull, false place a flower
        """
        if self.inPlay:
            inVec=np.reshape(self.gameState.state,(-1,1))
            inVec=np.concatenate((inVec,np.array([[rand()],[1]])))

            skull = ((1/(1+np.exp(-np.dot(self.theta_tp,inVec))))>=0.5)[0][0]

            if skull:
                self.placeSkull()
            else:
                self.placeFlower()
        else:
            raise NameError("Player is not in play")
            
    def betOrPlace(self): 
        """
        Function to decide to start the betting or place another tile
        Same input variables as choosing what to place and 
        -include if they have placed a skull already (binary) [1] have placed a skull
        EXTRA
        - random float for "randomness"
        - bais term
        
        RETURNS
        boolean. true, make a bet, false, place a tile
        """
        if self.inPlay:
            if len(self.placedTiles)<self.totalTiles:
                inVec=np.reshape(self.gameState.state,(-1,1))
                inVec=np.concatenate((inVec,np.array([[int(any(self.placedTiles))],[rand()],[1]])))

                betOrPlace = ((1/(1+np.exp(-np.dot(self.theta_bp,inVec))))>=0.5)[0][0]

                if betOrPlace:
                    bet = self.whatToBet()
                    print("Player bets: {}".format(bet))
                    self.currentBet=bet
                    self.betrec.playerbet(self.recPosition,bet)
                    self.game.stage+=1
                else:
                    self.placeTile()
                    print("Tile placed")
            else:
                bet = self.whatToBet()
                print("Player bets: {}".format(bet))
                self.currentBet=bet
                self.betrec.playerbet(self.recPosition,bet)
                self.game.stage+=1
        else:
            pass
            
            
    def whatToBet(self):
        """
        Function to decide what to bet
        Uses the board state record and the current betRecord. Also uses if they player has placed a skull, random float, bias and a list of available bets
        """
        if self.inPlay:
            inVec=np.concatenate((np.reshape(self.gameState.state,(-1,1)),
                                  np.reshape(self.betrec.betRecord,(-1,1)),np.array([[int(any(self.placedTiles))],[rand()],[1]])))
            availableBets=np.zeros((1,12))
            availableBets[:,0:sum(np.array(self.gameState.state)[:,2])]=1
            currentBet=(np.where(self.betrec.betRecord[:,:-1].any(axis=0))[0])
            if currentBet.shape[0]==0:
                currentBet=0
            else:
                currentBet=max(currentBet)+1
            availableBets[:,0:currentBet]=0
            inVec=np.concatenate((inVec,availableBets.T))
            bet=np.argmax((1/(1+np.exp(-np.dot(self.theta_w2b,inVec))))*availableBets.T)+1
            self.bet=bet
            return bet
        else:
            raise NameError("Player is not in play")
    
    def betOrPass(self):
        """
        Function to decide to bet or pass
        Same input variables as choosing what to place and 
        -include if they have placed a skull already (binary) [1] have placed a skull
        EXTRA
        - random float for "randomness"
        - bais term
        
        RETURNS
        boolean. true, make a bet, false, place a tile
        """
        if self.inPlay & (self.game.stage==0) & (self.hasPassed==False):
            inVec=np.concatenate((np.reshape(self.gameState.state,(-1,1)),np.reshape(self.betrec.betRecord,(-1,1)),
                                  np.array([[int(any(self.placedTiles))],[rand()],[1]])))
            availableBets=np.zeros((1,12))
            availableBets[:,0:sum(np.array(self.gameState.state)[:,2])]=1
            currentBet=(np.where(self.betrec.betRecord.any(axis=0))[0])
            if currentBet.shape[0]==0:
                currentBet=0
            else:
                currentBet=max(currentBet)+1
            availableBets[:,0:currentBet]=0
            if availableBets.any():
                inVec=np.concatenate((inVec,availableBets.T))
                betOrPass=(1/(1+np.exp(-np.dot(self.theta_bpa,inVec))))>=0.5

                if betOrPass:
                    bet = self.whatToBet()
                    print("Player bets: {}".format(bet))
                    self.currentBet=bet
                    self.betrec.playerbet(self.recPosition,bet)

                else:
                    self.hasPassed=True
                    print("Player passes")
                    self.betrec.playerPass(self.recPosition)
                    if self.betrec.switch:
                        self.game.stage+=1
            else:
                self.hasPassed=True
                print("Player passes")
                self.betrec.playerPass(self.recPosition)
                if self.betrec.switch:
                        self.game.stage+=1
        else:
            if not(self.inPlay):
                pass
            elif not(self.game.stage==0):
                raise NameError('Not in the betting stage')
            
    def takeTurn(self): #TO FINISH
        if self.inPlay:
            if self.game.stage==-1:
                self.betOrPlace()
            elif self.game.stage==0:
                self.betOrPass()
            elif self.game.stage==1:
                self.pickRec.setPickRec(self.recPosition)
                self.seek()
            
    def seek(self):
        if self.inPlay & (self.game.stage==1) & (self.hasPassed==False):
            if any(self.placedTiles):
                print("Player {} skulls themself".format(self.recPosition))
                self.removeTile()
            else:
                toFind = self.currentBet - len(self.placedTiles)
                idList = list(range(0,self.recPosition))+list(range(self.recPosition+1,4))
                while toFind>0:
                    rowId = self.whoToPick()
                    pickedPlayer = idList[rowId]
                    print("picked player {}".format(pickedPlayer))
                    tile = self.pickRec.pickPlayer(pickedPlayer,rowId)
                    if tile:
                        break
                    else:
                        toFind+=-1
                        print("{} flowers left to find".format(toFind))
                if toFind>0:
                    print("player has been skulled")
                    self.removeTile()
                else:
                    print("player {} has won a point".format(self.recPosition))
                    if self.hasPoint:
                        print("player {} wins the game".format(self.recPosition))
                        self.game.winnerFound=True
                        self.game.winner=self
                    else:
                        self.hasPoint=True
                        self.game.resetRound()
                        
                

    def whoToPick(self): #TO FINISH
        """
        Function to decide whose top tile to turn over when i pick
        uses board state, bet rec (does not need self information for these)
        and self pick rec
        extra
         - num of tiles to go
         - bias
        """
        inVec=np.concatenate((np.reshape(self.gameState.state,(-1,1)),np.reshape(self.betrec.betRecord,(-1,1)),
                              np.reshape(self.pickRec.pickRecord,(-1,1)),np.array([[1]])))
        whoToPick = np.argmax((1/(1+np.exp(-np.dot(self.theta_w2p,inVec))))*np.array(self.pickRec.pickRecord)[:,-1].reshape(3,1))
        return whoToPick

In [383]:
class skullGame:
    def __init__(self,player1,player2,player3,player4):
        self.playerArray = [player1,player2,player3,player4]
        self.firstPlayerOfRound = random.randint(0,len(self.playerArray)-1)
        self.gameState = skullGameState(self.firstPlayerOfRound,*self.playerArray)
        self.betRec = skullBetRec(*self.playerArray)                               #RR
        self.stage = -1                                                            #RR
        self.pickRec = skullPickRec(*self.playerArray)                             #RR
        self.winnerFound=False
        self.winner=None
        for i in range(0,4):
            player = self.playerArray[i]
            player.recPosition=i
            player.gameState = self.gameState
            player.betrec = self.betRec
            player.game = self
            player.placeTile()
            player.pickRec = self.pickRec
    
    def resetRound(self):
        self.betRec = skullBetRec(*self.playerArray)
        self.stage = -1
        self.gameState.gatherInfo()
        self.pickRec = skullPickRec(*self.playerArray)
        for i in range(0,4):
            player = self.playerArray[i]
            player.betrec = self.betRec #potentially redundant
            player.resetRound()
            
        for i in range(0,4):
            player = self.playerArray[i]
            player.placeTile()
            
    def playGame(self):
        while not(self.winnerFound):
            pass
    
        
        
class skullGameState:
    """
    External info should have for each player:
        - If they have a point (binary)
        - How many tiles they have (numerical)
        - How many tiles they have placed (numerical)
        - If they have their skull (dual-binary) [1,0] skull, [0,0] unknown, [0,1] no skull 
        - their place in the turn order (numerical)
        - if they're out of the game (binary) [1] out, [0] in
        - optional (not implemented)
            - if they just lost a tile
            - if they just won a point
    """
    
    def __init__(self,firstPlayer,player1,player2,player3,player4):
        self.playerArray = [player1,player2,player3,player4]
        self.firstPlayer = firstPlayer
        self.state=[]
        self.gatherInfo()
        
        
    def gatherInfo(self):
        self.state=[]
        for i in range(0,4):
            player = self.playerArray[i]
            self.state.append([int(player.hasPoint),player.totalTiles,len(player.placedTiles),
                               int(player.externalHasSkull),int(player.externalHasNoSkull),
                               ((i-self.firstPlayer)%4),1-int(player.inPlay)])
    
    def updateFirstPlayer(self,player):
        self.firstPlayer = player

class skullBetRec:
    def __init__(self,player1,player2,player3,player4):
        self.playerArray = [player1,player2,player3,player4]
        self.betRecord=np.zeros((len(self.playerArray),(len(self.playerArray)*3)+1))
        self.switch = False
        for i in range(0,4):
            player = self.playerArray[i]
            self.betRecord[i,-1] = (1-int(player.inPlay))
            
    def playerbet(self,playerId,bet):
        self.betRecord[playerId,bet-1] = 1
    
    def playerPass(self,playerId):
        self.betRecord[playerId,-1] = 1
        if sum(self.betRecord[:,-1])==3:
            self.switch=True
        
class skullPickRec:
    def __init__(self,player1,player2,player3,player4):
        self.pickerId=None
        self.playerArray = [player1,player2,player3,player4]
        self.pickRecord=[]
        
    def setPickRec(self,pickId):
        self.pickerId=pickId
        for i in range(0,4):
            if i==self.pickerId:
                continue
            else:
                player = self.playerArray[i]
                self.pickRecord.append([0,len(player.placedTiles),0,int(player.inPlay)])
    
    def pickPlayer(self,playerId,rowId):
        pickedPlayer = self.playerArray[playerId]
        tile = pickedPlayer.placedTiles.pop
        self.adjustInfo(rowId)
        
        if tile:
            pickedPlayer.externalHasSkull=True
        elif self.pickRecord[rowId,2] ==pickedPlayer.totalTiles:
            pickedPlayer.externalHasNoSkull=True
        return tile
    
    def adjustInfo(self,rowId):
        self.pickRecord[rowId][0] = 1
        self.pickRecord[rowId][1] += -1
        self.pickRecord[rowId][2] += 1
        if self.pickRecord[rowId][1]==0:
            self.pickRecord[rowId][3]=0
        
            

In [384]:
demoPlayers=[skullPlayer() for x in range(0,4)]
demoGame=skullGame(*demoPlayers)

In [385]:
demoGame.gameState.state

[[0, 4, 1, 1, 0, 1, 0],
 [0, 4, 1, 1, 0, 2, 0],
 [0, 4, 1, 1, 0, 3, 0],
 [0, 4, 1, 1, 0, 0, 0]]

In [386]:
i = np.argmin(np.array(demoGame.gameState.state)[:,-2])

In [394]:
demoPlayers[i].takeTurn()
i=(i+1)%4

ValueError: shapes (3,73) and (93,1) not aligned: 73 (dim 1) != 93 (dim 0)

In [392]:
demoGame.gameState.state

[[0, 4, 1, 1, 0, 1, 0],
 [0, 4, 1, 1, 0, 2, 0],
 [0, 4, 1, 1, 0, 3, 0],
 [0, 4, 1, 1, 0, 0, 0]]

In [393]:
demoGame.betRec.betRecord

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])